In [1]:
import pyspark
from datetime import datetime
from pyspark.sql.types import IntegerType, FloatType
from pyspark.sql.functions import udf,lit

from pyspark.sql import SQLContext

sc = pyspark.SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

# Queries:
1. the percentage of canceled flights per day, throughout the entire data set
2. weekly percentages of delays that are due to weather, throughout the entire data set 
3. the percentage of flights belonging to a given "distance group" that were able to halve their departure delays by the time they arrived at their destinations. Distance groups assort flights by their total distance in miles. Flights with distances that are less than 200 miles belong in group 1, flights with distances that are between 200 and 399 miles belong in group 2, flights with distances that are between 400 and 599 miles belong in group 3, and so on. The last group contains flights whose distances are between 2400 and 2599 miles.
4. a weekly "penalty" score for each airport that depends on both the its incoming and outgoing flights. The score adds 0.5 for each incoming flight that is more than 15 minutes late, and 1 for each outgoing flight that is more than 15 minutes late.


## 1. "The percentage of canceled flights per day, throughout the entire data set"


In [2]:
def getCancelledPercentage(file):
    d = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true')\
        .load(file)
    
    d_ymdc= d.select(d["Year"],d["Month"],d["DayofMonth"],d["Cancelled"])
    
    d_cancelled = d_ymdc.filter(d["Cancelled"]==1) \
        .groupBy(d["Year"],d["Month"],d["DayofMonth"]) \
        .count().withColumnRenamed("count","cancelled")
    
    d_total = d_ymdc \
        .groupBy(d["Year"],d["Month"],d["DayofMonth"]) \
        .count().withColumnRenamed("count","total")
    
    res = d_cancelled.join(d_total, on=[d_cancelled.Year == d_total.Year, d_cancelled.Month == d_total.Month,d_cancelled.DayofMonth == d_total.DayofMonth]) \
        .select(d_cancelled.Year,d_cancelled.Month,d_cancelled.DayofMonth,d_cancelled["cancelled"],d_total["total"])
    
    return res.withColumn("percentageCancelled", (res.cancelled/res.total))\
        .drop("cancelled","total")\
        .orderBy("Year","Month","DayofMonth")
        

res1=[getCancelledPercentage("../BDdata/"+str(i)+".csv") for i in range(1994,2009)]


In [3]:
def saveUnited(filename, res1):
    res=res1[0]
    for el in res1[1:]:
        res=res.union(el)
    res.coalesce(1).write.csv(filename,header=True)

saveUnited("./results/query1.csv",res1)

## Extra query: Weekly percentages of flights cancelled that are due to weather, throughout the entire data set 

In [8]:

def getWeatherPercentage(year, newYearFirstWeek=None):
    d = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true')\
        .load('../BDdata/'+year+'.csv')
    dr = d.filter(d["Cancelled"]=='1')\
        .select(d["Year"],d["Month"],d["DayofMonth"],d["CancellationCode"])

    d_cancelled_weather = dr.filter(d["CancellationCode"]=='B')

    d2 = dr.groupBy(d["Year"],d["Month"],d["DayofMonth"])\
        .count()\
        .withColumnRenamed("count","cancelled")
    
    d3 = d_cancelled_weather.groupBy(d["Year"],d["Month"],d["DayofMonth"])\
        .count()\
        .withColumnRenamed("count","weather")
    
    d4 = d2.join(d3, on=[d2["Year"]==d3["Year"],d2["Month"]==d3["Month"],d2["DayofMonth"]==d3["DayofMonth"]])\
        .select(d2["Year"],d2["Month"],d2["DayofMonth"],d2["cancelled"],d3["weather"])\
        .orderBy(d2["Year"],d2["Month"],d2["DayofMonth"])
        
    d5 = d4.withColumn("week", week(d4["Year"],d4["Month"],d4["DayofMonth"]))
    
    newYearFirstWeek1 = d5.filter(d5["week"]==1).filter(d5["Month"]=="12")
    if newYearFirstWeek is not None:
        d5 = d5.union(newYearFirstWeek)
    d5a = d5.filter((d5["week"]!=1) | (d5["Month"]!="12")).groupBy("Year","week")\
        .agg({"cancelled":"sum","weather":"sum"})
    d6 = d5a.withColumn("percentage",d5a["sum(weather)"]/d5a["sum(cancelled)"])\
        .select(d5a["Year"],d5a["week"],"percentage")\
        .sort("Year","week")
    return (d6, newYearFirstWeek1)

## 2. Weekly percentages of delays that are due to weather, throughout the entire data set 

In [10]:
def getWeatherDelayPercentage(start, end):
    d = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true')\
        .load('../BDdata/'+str(start)+'.csv')
    for i in range(start+1,end+1):
        d = d.union(sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true')\
                .load('../BDdata/'+str(i)+'.csv'))
    d1 = d.where("ArrDelay >= 0").select(d["Year"],d["Month"],d["DayofMonth"],d["ArrDelay"],d["WeatherDelay"])
    # needed check on presence of "NA" because otherwise an empty table is returned if filtered when no "NA" is present
    if d1.where("WeatherDelay == 'NA'").count() > 0:
        d1 = d1.where("WeatherDelay != 'NA'")

    d3 = d1.groupBy(d["Year"],d["Month"],d["DayofMonth"]).agg({"ArrDelay":"sum","WeatherDelay":"sum"})
    
    week = udf(lambda *x : datetime(x[0],x[1],x[2]).isocalendar()[1], IntegerType())
    year = udf(lambda *x : datetime(x[0],x[1],x[2]).isocalendar()[0], IntegerType())

    d4 = d3.withColumn("Week", week(d3["Year"],d3["Month"],d3["DayofMonth"]))\
        .withColumn("Year",year(d3["Year"],d3["Month"],d3["DayofMonth"])).drop(d3["Year"])

    d5 = d4.groupBy("Year","Week")\
            .agg({"sum(WeatherDelay)":"sum","sum(ArrDelay)":"sum"})
    d6 = d5.withColumn("percentage",d5["sum(sum(WeatherDelay))"]/d5["sum(sum(ArrDelay))"])\
            .drop("sum(sum(WeatherDelay))","sum(sum(ArrDelay))")\
            .sort("Year","Week")
    
    return d6
            
d7 = getWeatherDelayPercentage(1994,2008)
d7.coalesce(1).write.csv("./results/query2.csv",header=True)

## 3. The percentage of flights belonging to a given "distance group" that were able to halve their departure delays by the time they arrived at their destinations. 

Distance groups assort flights by their total distance in miles. Flights with distances that are less than 200 miles belong in group 1, flights with distances that are between 200 and 399 miles belong in group 2, flights with distances that are between 400 and 599 miles belong in group 3, and so on. The last group contains flights whose distances are between 2400 and 2599 miles.

In [11]:

def getPercentageFastDistanceGroup(init,end):
    d = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true')\
        .load('../BDdata/'+str(init)+'.csv')
    for i in range(init+1,end+1):
        d = d.union(sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true')\
            .load('../BDdata/'+str(i)+'.csv'))

    d1 = d.where(d["Cancelled"]==0)\
        .where(d["DepDelay"] != "NA")\
        .where(d["ArrDelay"] != "NA")\
        .where(d["Distance"] != "NA")\
        .select(d["DepDelay"],d["ArrDelay"],d["Distance"])
    
    distanceGroup = udf(lambda x : int(x)//200+1,IntegerType())
    d2 = d1.withColumn("DistanceGroup",distanceGroup(d1["Distance"]))\
        .drop(d1["Distance"])
    
    d3 = d2.where("DepDelay > 0")\
        .filter(d2["DepDelay"]>=2*d2["ArrDelay"])\
        .groupBy(d2.DistanceGroup).agg({"*":"count"})

    d4 = d2.groupBy(d2.DistanceGroup).agg({"*":"count"})
    
    d5 = d4.join(d3,on=[d3.DistanceGroup==d4.DistanceGroup])\
        .withColumn("percentage", d3["count(1)"]/d4["count(1)"])\
        .drop("count(1)")\
        .select(d4.DistanceGroup,"percentage").sort("DistanceGroup")
    return d5

res4 = getPercentageFastDistanceGroup(1994,2008)
res4.coalesce(1).write.csv("./results/query3.csv",header=True)

In [12]:
import pyspark.sql.functions as sf

def getPercentageFastDistanceGroup(init,end):
    d = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true')\
        .load('../BDdata/'+str(init)+'.csv')
    for i in range(init+1,end+1):
        d = d.union(sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true')\
            .load('../BDdata/'+str(i)+'.csv'))

    d1 = d.where(d["Cancelled"]==0)\
        .where(d["DepDelay"] != "NA")\
        .where(d["ArrDelay"] != "NA")\
        .where(d["Distance"] != "NA")
    d1 = d1.select(d["Origin"],d["Dest"],d["Distance"])
    
    distanceGroup = udf(lambda x : int(x)//200+1,IntegerType())
    d2 = d1.withColumn("DistanceGroup",distanceGroup(d1["Distance"]))\
        .drop(d1["Distance"])
    
    d3 = d2.groupBy(d2.Origin, d2.Dest, d2.DistanceGroup).agg(sf.count("*").alias("Num"))
    d4a = d3.select(d3.Origin, d2.DistanceGroup, d3.Num).groupBy(d3.Origin,d3.DistanceGroup).agg(sf.sum(d3.Num).alias("Out"))
    d4b = d3.select(d3.Dest, d2.DistanceGroup, d3.Num).groupBy(d3.Dest,d3.DistanceGroup).agg(sf.sum(d3.Num).alias("In"))
    d5 = d4a.join(d4b, on=[d4a.Origin == d4b.Dest, d4a.DistanceGroup == d4b.DistanceGroup])\
        .select(d4a.DistanceGroup, d4a.Origin, d4a.Out, d4b.In)
    d5 = d5.withColumn("Total", d5.Out + d5.In)\
        .orderBy("DistanceGroup", "Origin")
        
    return d5

res4 = getPercentageFastDistanceGroup(1994,2008)
res4.coalesce(1).write.csv("./results/query3.csv/extra",header=True)

## 4. A weekly "penalty" score for each airport that depends on both the its incoming and outgoing flights. 
The score adds 0.5 for each incoming flight that is more than 15 minutes late, and 1 for each outgoing flight that is more than 15 minutes late.

In [13]:

week = udf(lambda *x : datetime(x[0],x[1],x[2]).isocalendar()[1], IntegerType())
year = udf(lambda *x : datetime(x[0],x[1],x[2]).isocalendar()[0], IntegerType())
penalty = lambda y : udf(lambda x: y if int(x) > 15 else 0.0, FloatType())


In [14]:

def getAirportPenalty(start, end):
    
    d = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true')\
        .load('../BDdata/'+str(start)+'.csv')
    for i in range(start+1,end+1):
        d = d.union(sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true')\
                .load('../BDdata/'+str(i)+'.csv'))
    d1 = d.withColumn("Week", week(d["Year"],d["Month"],d["DayofMonth"]))\
        .withColumn("Year", year(d["Year"],d["Month"],d["DayofMonth"])).drop(d["Year"])
    d1  = d1.filter((d1['DepDelay']!= "NA") & (d1['ArrDelay'] != "NA") & ((d1['ArrDelay'] > 15) | (d1['DepDelay'] > 15)))
    d2 = d1.withColumn("in", penalty(0.5)(d1['ArrDelay']))\
        .withColumn("out", penalty(1.0)(d1['DepDelay']))\
        .select(d1["Year"],d1["Week"],d1["Origin"],d1["Dest"],"in","out")
    d3 = d2.groupBy(d2["Year"],d2["Origin"],d2["Week"])\
        .agg({"out":"sum"})
    d3a = d2.groupBy(d2["Year"],d2["Dest"],d2["Week"]).agg({"in":"sum"})
    d3b = d3a.join(d3, on=[d3a["Dest"]==d3["Origin"],d3a["Year"]==d3["Year"],d3a["Week"]==d3["Week"]])\
            .select(d3["Year"],d3["Week"],d3["sum(out)"],d3a["sum(in)"],d3["Origin"])
    
    d4 = d3b.withColumn("Penalty", d3b["sum(out)"]+d3b["sum(in)"])\
        .drop("Dest")\
        .withColumnRenamed("Origin","Airport")\
        .withColumnRenamed("sum(out)","OutgoingPenalty")\
        .withColumnRenamed("sum(in)","IngoingPenalty")\
        .sort("Penalty")
    return d4

res5 = getAirportPenalty(1994,2008)
res5.coalesce(1).write.csv("./results/query4.csv",header=True)